In [1]:
!pip install tavily-python
!pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-t

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
tvly_api_key = user_secrets.get_secret("tvly_api_key")
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

In [3]:
from tavily import TavilyClient
from langchain_openai import ChatOpenAI

In [4]:
def get_urls(topic):
    """
    Fetch URLs related to a specific topic using the TavilyClient API.

    This function takes a topic as input, creates an instance of TavilyClient 
    with the specified API key, and performs a search for the topic. It 
    returns a list of URLs from the search results.

    Args:
        topic (str): The topic or keyword to search for.

    Returns:
        list: A list of URLs (str) related to the specified topic.
    """
    client = TavilyClient(api_key=tvly_api_key)
    response = client.search(topic)

    return [tvly_result["url"] for tvly_result in response["results"]]

urls = get_urls("India")
print(urls)

['https://simple.wikipedia.org/wiki/India', 'https://www.britannica.com/place/India/People', 'https://www.cia.gov/the-world-factbook/countries/india/', 'https://www.britannica.com/place/India/History', 'https://en.wikipedia.org/wiki/India']


In [5]:
llm = ChatOpenAI(api_key =OPENAI_API_KEY)
# llm.invoke("Hello, world!")